In [3]:
import numpy as np
#from load_data import load_mnist
from sklearn.datasets import load_digits
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
from sklearn.metrics import pairwise_distances
import seaborn as sns

#palette=sns.color_palette("cubehelix", 10)
palette=sns.color_palette("magma", 10)
#palette=sns.color_palette("viridis", 10)

In [ ]:
#Load the data
X, y = load_digits(return_X_y=True)

## sklearn implementation

In [ ]:
#Implement tsne from sklearn
tsne = TSNE()
X_embedded = tsne.fit_transform(X)

In [ ]:
X_embedded

In [ ]:
#Plots

def plot_tsne(xy, hue='hue', legend = "full", palette = palette):
    sns.set(rc={'figure.figsize':(10,10)})
    #palette=sns.color_palette("cubehelix", 10)
    palette=sns.color_palette("magma", 10)
    #palette=sns.color_palette("viridis", 10)
    fig = sns.scatterplot(xy[:,0], xy[:,1],
                hue = hue,
                legend = "full",
                palette = palette) # don't use edges
    plt.show()
    
plot_tsne(X_embedded, hue = y)

## My implementation

Step 1: Euclidian distances

In [4]:
def pair_distances(X):
    D_squared = np.sum((X[None,:] - X[:, None])**2, -1)#**0.5 #OJO AQUI CON LA RAIZ
    return -D_squared

In [2]:
X = np.array([(0,0), (4,0), (4,3), (0,3)])
print(X)
#-1*pairwise_distances(X, metric='euclidean', squared=True)==pair_distances(X)
pair_distances(X)

NameError: name 'np' is not defined

In [ ]:
e_x=np.exp(X - np.max(X, axis=1).reshape([-1, 1]))
print(e_x)

In [ ]:
np.fill_diagonal(e_x, 0.)
print(e_x)

Step 2: Compute the quotient and the probability matrix

In [5]:
def quotient(distances):
    # Subtract max for numerical stability
    e_x = np.exp(distances - np.max(distances, axis=1).reshape([-1, 1]))

    # Diagonal 0.
    np.fill_diagonal(e_x, 0.)

    # Add a tiny constant for stability of log we take later
    e_x = e_x + 1e-8  # numerical stability

    return e_x / e_x.sum(axis=1).reshape([-1, 1])

In [6]:
def compute_proba_matrix(distances,sigmas=None):
    if sigmas==None: 
        return quotient(distances)
    else:
        return quotient(distances / 2**np.square(sigmas.reshape((-1, 1))))
        

Step 3: Compute the perplexity if needed

In [7]:
#First a binary search 
def binary_search(array,target,tol=1e-10,max_iter=1000,lower=1e-20,upper=1000.):
    for i in range(max_iter):
        guess = ((lower + upper) / 2.) + lower
        val = array(guess)
        if val > target:
            upper = guess
        else:
            lower = guess
        if np.abs(val - target) <= tol:
            break
    return guess

In [8]:
def calc_perplex(distances,sigmas):
    proba_matrix=compute_proba_matrix(distances,sigmas)
    logar=np.log2(proba_matrix)
    entropy = -np.sum(proba_matrix*logar,1)
    perplexity = 2**entropy
    return perplexity

In [9]:
#SNE performs a binary searh for the value of sigma_i that ptoduces a P_i with a
#fixed perplexity that is specified by the user
def find_sigmas(distances,our_perplexity):
    sigmas=np.zeros(distances.shape[0])
    # For each row of the matrix (each point in our dataset)
    for i in range(distances.shape[0]):
        # Make fn that returns perplexity of this row given sigma
        eval_fn = lambda sigma: \
            calc_perplex(distances[i:i+1, :], np.array(sigma))
        # Binary search over sigmas to achieve target perplexity
        correct_sigma = binary_search(eval_fn, our_perplexity,tol=1e-10,max_iter=1000,lower=1e-20,upper=1000.)
        # Save the correct sigma 
        sigmas[i]=correct_sigma
    return np.array(sigmas)

In [10]:
def solve_crowd(P):
    N = P.shape[0]
    return (P+P.T)/2*N

In [11]:
#Compute the low dimesional map qij using ec 4
def Q_table(Y):
    distances_low_dim=pair_distances(Y)
    nom = np.linalg.inv(np.exp(1.-distances))
    # Fill diagonal with zeroes so q_ii = 0
    np.fill_diagonal(exp_distances, 0.) # WARNING: NUMERICAL STABILITY
    deno = np.linalg.inv(np.sum(1.-exp_distances))
    return nom/den
    

In [12]:
def P_table(X,our_perplexity):
    distances_high_dim = pair_distances(X)
    sigmas=find_sigmas(distances_high_dim,our_perplexity)
    compute_proba_matrix(distances_high_dim,sigmas)
    return solve_crowd(P)

In [13]:
def compute_gradient(P,Q,Y):
    
    pq_diff = P - Q
    pq_expanded = np.expand_dims(pq_diff, 2)
    y_diffs = np.expand_dims(Y, 1) - np.expand_dims(Y, 0)
    
    distances_low_dim=pair_distances(Y)
    aux = np.linalg.inv(np.exp(1.-distances))
    distances_low_expanded = np.expand_dims(aux, 2)
    
    y_diffs_wt = y_diffs * distances_low_expanded

    # Multiply then sum over j's
    grad = 4. * (pq_expanded * y_diffs_wt).sum(1)
    
    return grad


In [14]:
def t_sne(X,classes,our_perplexity, num_iter, nu, alpha):
    
    N = X.shape[0]
    P = P_table(X,our_perplexity)
    random.seed(10)
    Y = np.random.normal(loc=0.0, scale=1e-4, size=(N,2))
    Y_m2 = Y.copy()
    Y_m1 = Y.copy()
    for i in range(T):
        Q= Q_table(Y) #compute low dimensional affinities
        grad = compute_gradient(P,Q,Y) # compute gradient
        Y = Y + nu*grad + alpha*(Y_m1-Y_m2)#set y_i's
        Y_m2 = Y_m1.copy()
        Y_m1 = Y.copy()
    
    return Y #low dimensional representation

In [15]:
      # Set global parameters
NUM_POINTS = 100            # Number of samples from MNIST
CLASSES_TO_USE = [0, 1, 8]  # MNIST classes to use
PERPLEXITY = 20                   # Random seed
MOMENTUM = 0.9              #alpha  
LEARNING_RATE = 10.         # nu
NUM_ITERS = 10             # Num iterations to train for  


# numpy RandomState for reproducibility
rng = np.random.RandomState(1)

# Load the first NUM_POINTS 0's, 1's and 8's from MNIST
#X, y = load_mnmist('datasets/',
#                  digits_to_keep=CLASSES_TO_USE,#                  N=NUM_POINTS)

X, y = load_digits(return_X_y=True)
X = X[0:200]

# Obtain matrix of joint probabilities p_ij
#P = p_joint(X, PERPLEXITY)

# Fit SNE or t-SNE
X_embeded = t_sne(X=X,classes=y,our_perplexity=PERPLEXITY, num_iter=NUM_ITERS, 
          nu=LEARNING_RATE, alpha=MOMENTUM)

#Y = estimate_sne(X, y, P, rng,
#         num_iters=NUM_ITERS,
#         q_fn=q_tsne if TSNE else q_joint,
#         grad_fn=tsne_grad if TSNE else symmetric_sne_grad,
#         learning_rate=LEARNING_RATE,
#         momentum=MOMENTUM,
#         plot=NUM_PLOTS)                                                                  

<ipython-input-6-1325132c1845>:5: RuntimeWarning: overflow encountered in power
  return quotient(distances / 2**np.square(sigmas.reshape((-1, 1))))


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
X